<h1>Drug Safety Test</h1>

<h2>Import Libraries</h2>

In [130]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import ttest_ind

<h2>Task 1</h2>

In [148]:
df = pd.read_csv("drug_safety.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16103 entries, 0 to 16102
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              16103 non-null  int64  
 1   sex              16103 non-null  object 
 2   trx              16103 non-null  object 
 3   week             16103 non-null  int64  
 4   wbc              9128 non-null   float64
 5   rbc              9127 non-null   float64
 6   adverse_effects  16103 non-null  object 
 7   num_effects      16103 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 1006.6+ KB


<h2>Task 2</h2>

In [149]:
size = len(df)
size = int(size * 0.1)
nan_positions = list(zip(*np.where(df.isnull())))
num_to_remove = min(size, len(nan_positions))
np.random.seed(42)
positions_to_remove = np.random.choice(len(nan_positions), num_to_remove, replace=False)
df = df.drop(positions_to_remove)
df_clean = df.fillna(df.median(numeric_only=True))
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14493 entries, 0 to 16102
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              14493 non-null  int64  
 1   sex              14493 non-null  object 
 2   trx              14493 non-null  object 
 3   week             14493 non-null  int64  
 4   wbc              14493 non-null  float64
 5   rbc              14493 non-null  float64
 6   adverse_effects  14493 non-null  object 
 7   num_effects      14493 non-null  int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 1019.0+ KB


<h2>Task 3</h2>

In [150]:
stats = df_clean.describe().loc[['mean', 'std']]
print(stats)

            age      week       wbc       rbc  num_effects
mean  64.080660  7.735458  7.235383  4.685147     0.101773
std    8.828372  6.934096  1.509928  0.345739     0.322460


<h2>Task 2 & 4</h2>

In [151]:
cleaned_df.dropna(inplace=True)
grouped_stats = df_clean.groupby("trx")
grouped_stats2 = df.groupby("trx")
grouped_stats.describe()

age                                   ... num_effects                    
          count       mean       std   min   25%  ...         min  25%  50%  75%  max
trx                                               ...                                
Drug     9625.0  64.151688  8.793976  39.0  58.0  ...         0.0  0.0  0.0  0.0  3.0
Placebo  4868.0  63.940222  8.895226  39.0  58.0  ...         0.0  0.0  0.0  0.0  3.0

[2 rows x 40 columns]

<h2>Task 5</h2>

In [152]:
df_clean['adverse_effects'] = df_clean['adverse_effects'].map({'Yes': 1, 'No': 0})
grouped_stats = df_clean.groupby("trx")
grouped_stats.describe()

age                                   ... num_effects                    
          count       mean       std   min   25%  ...         min  25%  50%  75%  max
trx                                               ...                                
Drug     9625.0  64.151688  8.793976  39.0  58.0  ...         0.0  0.0  0.0  0.0  3.0
Placebo  4868.0  63.940222  8.895226  39.0  58.0  ...         0.0  0.0  0.0  0.0  3.0

[2 rows x 48 columns]

<h2>Task 6</h2>

In [153]:
df = df.dropna()
drug_group = df_clean[df_clean['trx'] == 'Drug']
placebo_group = df_clean[df_clean['trx'] == 'Placebo']

metrics = ['wbc', 'rbc', 'num_effects', 'adverse_effects']
results = {}

for metric in metrics:
    t_stat, p_value = ttest_ind(drug_group[metric], placebo_group[metric], alternative='two-sided', equal_var=False)
    results[metric] = {'t-statistic': t_stat, 'p-value': p_value}

for metric, result in results.items():
    print(f"Metric: {metric}")
    print(f"T-statistic: {result['t-statistic']}")
    print(f"P-value: {result['p-value']}")
    if result['p-value'] < 0.05:
        print("Conclusion: Reject the null hypothesis for %5 p-value(significant difference).")
    else:
        print("Conclusion: Fail to reject the null hypothesis for %5 p-value(no significant difference).")
    if result['p-value'] < 0.1:
        print("Conclusion: Reject the null hypothesis for %10 p-value(significant difference).")
    else:
        print("Conclusion: Fail to reject the null hypothesis for %10 p-value(no significant difference).")

Metric: wbc
T-statistic: -1.1686996473299733
P-value: 0.24255357791472182
Conclusion: Fail to reject the null hypothesis for %5 p-value(no significant difference).
Conclusion: Fail to reject the null hypothesis for %10 p-value(no significant difference).
Metric: rbc
T-statistic: 1.429391931649759
P-value: 0.15292484963199635
Conclusion: Fail to reject the null hypothesis for %5 p-value(no significant difference).
Conclusion: Fail to reject the null hypothesis for %10 p-value(no significant difference).
Metric: num_effects
T-statistic: 1.0223476834318055
P-value: 0.30664055818306846
Conclusion: Fail to reject the null hypothesis for %5 p-value(no significant difference).
Conclusion: Fail to reject the null hypothesis for %10 p-value(no significant difference).
Metric: adverse_effects
T-statistic: 0.5728950075542752
P-value: 0.5667288074294856
Conclusion: Fail to reject the null hypothesis for %5 p-value(no significant difference).
Conclusion: Fail to reject the null hypothesis for %10 p

<b>If we raise significant level it means that we want to reduce the type II error or type II error is more prior</b>

<b>`alternative` argument specifies the type of alternative hypothesis testing in a t-test.<br>It can be `two-sided`, `less` and `greater`</b>

<b>The `equal_var` argument specifies whether the two groups being compared have equal variances.<br>if we set it `True` it means the two groups have equal population variances.<br>If we set it `False` it means the two groups have unequal variances.
</b>